In [1]:
import pickle
import time

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm import tqdm

from astropy.table import Table

In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [3]:
data_dir = 'data'
with open(f'{data_dir}/features_sdssdr16+psdr2+all_deacls8tr_QSO+GALAXY_20201212133711.pkl', 'rb') as f:
    features_list = pickle.load(f)
models_cols = ['max19_z', 'conf19_z', 'max21_z', 'conf21_z', 'max22_z', 'conf22_z', 'max35_z', 'conf35_z']

In [4]:
np.random.seed(0)
torch.manual_seed(0)

In [5]:
train_20 = pd.read_csv(f'{data_dir}/q_photo_z_train_20.csv')

In [6]:
train_20

,RA,DEC,Z,sdssdr16_u_psf,sdssdr16_g_psf,sdssdr16_r_psf,sdssdr16_i_psf,sdssdr16_z_psf,sdssdr16_u_cmodel,sdssdr16_i_cmodel,...,sdssdr16_z_cmodel-decals8tr_z,max19_z,conf19_z,max21_z,conf21_z,max22_z,conf22_z,max35_z,conf35_z,fold
0,0.000722,11.343983,0.442757,23.376900,22.425905,20.799930,20.031074,19.622680,20.226514,19.305746,...,0.434388,0.474689,0.928341,0.458632,0.936633,0.419659,0.636504,0.447622,0.961682,0
1,0.001417,18.492306,0.629820,22.113601,22.968215,21.524378,20.363373,19.827644,22.126405,19.405817,...,-0.151581,0.599155,0.926998,0.647224,0.999999,0.708078,0.910233,0.604697,0.955222,0
2,0.001885,17.773712,2.309000,22.265783,21.813904,21.999912,21.870190,21.266889,22.343590,21.853381,...,-0.287098,2.322151,0.312809,2.300145,0.422922,2.368000,0.576355,2.121082,0.503914,1
3,0.002416,5.941882,2.103120,22.059562,21.434419,21.211019,21.184635,20.688257,22.046778,21.198729,...,-0.543980,0.846000,0.331083,0.819000,0.382401,0.916635,0.653894,0.801000,0.485579,1
4,0.002769,14.974691,2.497000,21.761608,21.095956,20.797531,20.715893,20.589152,21.633194,20.676625,...,-0.086858,2.320000,0.558577,2.482500,0.664471,2.625000,0.634396,2.404000,0.824371,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580451,359.998949,10.116818,2.415000,23.056658,21.692741,21.373058,21.151950,21.260001,21.525019,21.153138,...,0.502369,2.343000,0.448830,2.362000,0.289139,2.620000,0.275886,2.598000,0.508071,0
580452,359.999026,24.413551,1.490065,20.921111,20.732991,20.529058,20.355686,20.119849,20.765828,20.362367,...,-0.534182,1.696801,0.276569,1.516000,0.353289,1.471000,0.272345,1.519121,0.532895,0
580453,359.999121,28.954727,2.452000,21.876842,21.168578,21.233727,21.235761,20.713764,21.639407,21.276328,...,-0.513353,2.360000,0.368830,2.349000,0.583509,2.416101,0.459684,2.333000,0.827930,1
580454,359.999634,3.268618,1.233161,18.791953,18.815835,18.557703,18.614296,18.673656,18.798606,18.611784,...,-0.625142,1.261000,0.765830,1.264800,0.803640,1.163000,0.854128,1.319833,0.751813,0


In [7]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [8]:
class SimpleDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        if self.X.shape[0] != self.y.shape[0]:
            raise ValueError
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [9]:
class MLP(nn.Module):
    def __init__(self, sizes, p):
        super().__init__()
        self.sizes = sizes
        layers = []
        for i in range(1, len(sizes)):
            if i > 1:
                layers.append(nn.ReLU())
                layers.append(nn.Dropout(p))
            layers.append(nn.Linear(self.sizes[i-1], self.sizes[i], bias=False))
            if i < len(sizes) - 1:
                layers.append(nn.BatchNorm1d(self.sizes[i]))
        self.layers = nn.Sequential(*layers)
        self.layers.apply(self.init_weights)
        
    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
                          
    def forward(self, x, train_m):
        self.train(train_m)
        out = self.layers(x)
        mean, var = out[:, 0], out[:, 1]
        return mean, var
    
    def run_epoch(self, dataloader, optimizer=None, loss_f=None):
        train_m = (optimizer is not None) and (loss_f is not None)
        true, mean, var = [], [], []
        for X, y in dataloader:
            if train_m:
                optimizer.zero_grad()
            batch_mean, batch_var = self(X, train_m)
            # batch_out = self(X)
            # batch_mean, batch_var = batch_out[:, 0], batch_out[:, 1]
            if train_m:
                loss = loss_f(y, batch_mean, batch_var)
                loss.backward()
                optimizer.step()
            true.append(y.cpu().detach().numpy().reshape(-1))
            mean.append(batch_mean.cpu().detach().numpy().reshape(-1))
            var.append(batch_var.cpu().detach().numpy().reshape(-1))
        true = np.concatenate(true)
        mean = np.concatenate(mean)
        var = np.concatenate(var)
        return true, mean, var
                          
#     def fit(self, train_dataloader, test_dataloader, epochs, optimizer, loss_f, metrics=[]):
#         for epoch in range(epochs):
#             print(f'Epoch #{epoch+1}/{epochs}')
#             start = time.time()
#             train_loss, test_loss = 0, 0
#             batch_num = 0
#             preds = []
#             true = []
#             for X, y in train_dataloader:
#                 batch_num += 1
#                 optimizer.zero_grad()
#                 outputs = self(X)
#                 mean, var = outputs[:, 0], outputs[:, 1]
#                 loss = loss_f(y, mean, var)
#                 train_loss += loss.item()
#                 loss.backward()
#                 optimizer.step()
#                 preds.append(mean.cpu().detach().numpy().reshape(-1))
#                 true.append(y.cpu().detach().numpy().reshape(-1))
#             train_loss /= batch_num
#             print(f'Train loss: {round(train_loss, 5)}\t', end='')
            
#             preds = np.concatenate(preds)
#             true = np.concatenate(true)
#             for metric in metrics:
#                 print(f'Train {metric}: {round(metrics[metric](true, preds), 5)}\t', end='')
#             print()
                
#             batch_num = 0
#             preds = []
#             true = []
#             for X, y in test_dataloader:
#                 batch_num += 1
#                 outputs = self(X)
#                 mean, var = outputs[:, 0], outputs[:, 1]
#                 loss = loss_f(y, mean, var)
#                 test_loss += loss.item()
#                 preds.append(mean.cpu().detach().numpy().reshape(-1))
#                 true.append(y.cpu().detach().numpy().reshape(-1))
#             test_loss /= batch_num
#             print(f'Test  loss: {round(test_loss, 5)}\t', end='')
            
#             preds = np.concatenate(preds)
#             true = np.concatenate(true)
#             for metric in metrics:
#                 print(f'Test  {metric}: {round(metrics[metric](true, preds), 5)}\t', end='')
#             print()
#             print(f'Time: {round(time.time() - start, 5)}')
#             print('=' * 30)
    
    @classmethod
    def get_dataloader(cls, X, y, batch_size=2**13, shuffle=True):
        order = np.arange(X.shape[0], dtype=int)
        if shuffle:
            np.random.shuffle(order)
        dataset = SimpleDataset(X, y)
        dataloader = DataLoader(dataset, batch_size, shuffle=False)
        return dataloader

In [10]:
class DeepEnsemble:
    def __init__(self, BaseModel, base_model_args, M, device=torch.device('cpu')):
        self.BaseModel = BaseModel
        self.base_model_args = base_model_args
        self.M = M
        self.device = device
        self.models = []
        for i in range(M):
            self.models.append(self.BaseModel(**self.base_model_args).to(self.device))
        
    def loss(self, y, mean, var):
        if isinstance(y, torch.Tensor):
            return ((var + torch.exp(-var) * (mean - y)**2) / 2).mean()
        else:
            return ((var + np.exp(-var) * (mean - y)**2) / 2).mean()
                
    def fit(
        self, X_train, y_train, X_test=None, y_test=None, 
        epochs=5, batch_size=2**13, shuffle=True, 
        optimizer=torch.optim.Adam, optimizer_args={'lr': 0.0005, 'weight_decay': 0.0001}, 
        verbose=True, metrics=[]
    ):
        print(f'Device: {self.device}')
        X_train_tensor = torch.tensor(X_train, device=torch.device(device), dtype=torch.float)
        y_train_tensor = torch.tensor(y_train, device=torch.device(device), dtype=torch.float)
        X_test_tensor = torch.tensor(X_test, device=torch.device(device), dtype=torch.float)
        y_test_tensor = torch.tensor(y_test, device=torch.device(device), dtype=torch.float)
        
        test_dataloader = self.BaseModel.get_dataloader(
            X_test_tensor, y_test_tensor,
            batch_size=batch_size, shuffle=shuffle
        )
        
        test_m = (X_test is not None) and (y_test is not None)
        print(f'Test mode: {test_m}')
        print('=' * 40)
        
        optimizers = []
        for model in self.models:
            optimizers.append(optimizer(model.parameters(), **optimizer_args))

        train_metric_vals = {metric: [] for metric in metrics}
        train_losses = []
        if test_m:
            test_metric_vals = {metric: [] for metric in metrics}
            test_losses = []
        else:
            test_metric_vals = None
            test_losses = None
    
        for epoch in range(epochs):
            print(f'EPOCH #{epoch+1}/{epochs}:')
            print('-' * 40)
            start = time.time()
            
            #TRAIN
            
            train_dataloader = self.BaseModel.get_dataloader(
                X_train_tensor, y_train_tensor,
                batch_size=batch_size, shuffle=shuffle
            )
            
            epoch_mean, epoch_var = [], []
            epoch_losses = []
            for i, model in enumerate(self.models):
                epoch_true, m, v = model.run_epoch(train_dataloader, optimizers[i], self.loss)
                epoch_mean.append(m)
                epoch_var.append(v)
                epoch_losses.append(self.loss(epoch_true, m, v).item())
            t_m = np.array(epoch_mean)
            epoch_mean = np.mean(t_m, axis=0)
            t_v = np.array(epoch_var)
            epoch_var = np.mean(t_v, axis=0) + np.mean(t_m - epoch_mean, axis=0)
            
            train_losses.append(epoch_losses)
            if verbose:
                print(f'Trian losses: {[round(l, 5) for l in epoch_losses]}')
                print(f'AVG train loss: {round(np.mean(epoch_losses), 5)}')
            
            for metric in metrics:
                train_metric_vals[metric].append(metrics[metric](epoch_true, epoch_mean))    
            if verbose:
                for metric in metrics:
                      print(f'Train {metric}: {round(train_metric_vals[metric][-1], 5)}\t', end='')
                print()
            print('-' * 40)
                
            #TEST
            if test_m:     
                epoch_mean, epoch_var = [], []
                epoch_losses = []
                for i, model in enumerate(self.models):
                    epoch_true, m, v = model.run_epoch(test_dataloader)
                    epoch_mean.append(m)
                    epoch_var.append(v)
                    epoch_losses.append(self.loss(epoch_true, m, v).item())
                t_m = np.array(epoch_mean)
                epoch_mean = np.mean(t_m, axis=0)
                t_v = np.array(epoch_var)
                epoch_var = np.mean(t_v, axis=0) + np.mean(t_m - epoch_mean, axis=0)

                test_losses.append(epoch_losses)
                if verbose:
                    print(f'Test losses: {[round(l, 5) for l in epoch_losses]}')
                    print(f'AVG test loss: {round(np.mean(epoch_losses), 5)}')

                for metric in metrics:
                    test_metric_vals[metric].append(metrics[metric](epoch_true, epoch_mean))    
                if verbose:
                    for metric in metrics:
                          print(f'Test {metric}: {round(test_metric_vals[metric][-1], 5)}\t', end='')
                    print()
                print('-' * 40)
            
            print(f'Time: {round(time.time() - start, 3)}')
            print('=' * 40)
        return train_metric_vals, train_losses, test_metric_vals, test_losses

In [11]:
def sigma_nmad(true, preds):
    diff = preds - true
    m = np.median(diff)
    return 1.48 * np.median(np.abs((diff - m) / (1 + true)))

In [12]:
def out_rate(true, preds):
    diff = preds - true
    return ((np.abs(diff) / (1 + true)) > 0.15).sum() / len(true)

In [13]:
train_20_35_01 = train_20[train_20['fold'] == 0].dropna()[['RA', 'DEC', 'Z'] + features_list + ['max35_z', 'conf35_z']]
X_01, y_01 = train_20_35_01[features_list].values.astype(float), train_20_35_01['Z'].values.astype(float)
train_20_35_02 = train_20[train_20['fold'] == 1].dropna()[['RA', 'DEC', 'Z'] + features_list + ['max35_z', 'conf35_z']]
X_02, y_02 = train_20_35_02[features_list].values.astype(float), train_20_35_02['Z'].values.astype(float)

In [14]:
ens = DeepEnsemble(
    MLP, 
    {'sizes': [X_01.shape[1], 256, 256, 256, 256, 2], 'p': 0.0},
    10, device
)
optimizer = torch.optim.Adam

In [15]:
ens.fit(
    X_train=X_01, 
    y_train=y_01,
    X_test=X_02,
    y_test=y_02,
    epochs=100, 
    batch_size=2**15,
    shuffle=True,
    optimizer=optimizer, 
    optimizer_args={'lr': 0.0005, 'weight_decay': 0.001},
    verbose=True,
    metrics={'S_nmad': sigma_nmad, 'out_rate': out_rate}
);

Device: cuda:0
Test mode: True
EPOCH #1/100:
----------------------------------------
Trian losses: [1.64056, 0.38958, 0.70891, 1.61283, 1.38108, 0.89885, 255.02843, 2.44769, 2309.33301, 3.28835]
AVG train loss: 257.67293
Train S_nmad: 0.33198	Train out_rate: 0.94697	
----------------------------------------
Test losses: [11.50263, 2.85191, 6.61537, 3.98491, 5.29903, 3.74524, 5.81755, 1.8631, 1.9196, 2.80928]
AVG test loss: 4.64086
Test S_nmad: 0.47804	Test out_rate: 1.0	
----------------------------------------
Time: 63.168
EPOCH #2/100:
----------------------------------------
Trian losses: [0.69954, -0.01786, 0.25523, 0.55079, 0.72381, 0.41742, 2.34544, 0.46079, 1.65502, 0.70414]
AVG train loss: 0.77943
Train S_nmad: 0.28626	Train out_rate: 0.79282	
----------------------------------------
Test losses: [5.26351, 0.45423, 1.90966, 2.45757, 2.03233, 2.05639, 1.88344, 1.3178, 1.69179, 1.2053]
AVG test loss: 2.0272
Test S_nmad: 0.55758	Test out_rate: 0.95663	
---------------------------

In [16]:
ens.fit(
    X_train=X_01, 
    y_train=y_01,
    X_test=X_02,
    y_test=y_02,
    epochs=100, 
    batch_size=2**15,
    shuffle=True,
    optimizer=optimizer, 
    optimizer_args={'lr': 0.0001, 'weight_decay': 0.001},
    verbose=True,
    metrics={'S_nmad': sigma_nmad, 'out_rate': out_rate}
);

Device: cuda:0
Test mode: True
EPOCH #1/100:
----------------------------------------
Trian losses: [-1.43063, -1.47719, -1.3695, -1.29533, -1.46697, -1.53173, -0.76006, -1.18248, -0.7615, -1.37595]
AVG train loss: -1.26513
Train S_nmad: 0.04472	Train out_rate: 0.08231	
----------------------------------------
Test losses: [-1.40292, -1.42794, -1.32529, -1.24755, -1.39773, -1.44264, -0.79209, -1.16209, -0.78449, -1.34903]
AVG test loss: -1.23318
Test S_nmad: 0.04365	Test out_rate: 0.08296	
----------------------------------------
Time: 61.648
EPOCH #2/100:
----------------------------------------
Trian losses: [-1.44808, -1.43097, -1.32364, -1.3333, -1.43693, -1.4711, -0.81301, -1.18966, -0.80611, -1.38726]
AVG train loss: -1.264
Train S_nmad: 0.04386	Train out_rate: 0.0814	
----------------------------------------
Test losses: [-1.37924, -1.40868, -1.31851, -1.33984, -1.35343, -1.43212, -0.81844, -1.17757, -0.79847, -1.31533]
AVG test loss: -1.23416
Test S_nmad: 0.04169	Test out_rate:

In [ ]:
ens.fit(
    X_train=X_01, 
    y_train=y_01,
    X_test=X_02,
    y_test=y_02,
    epochs=100, 
    batch_size=2**15,
    shuffle=True,
    optimizer=optimizer, 
    optimizer_args={'lr': 0.00005, 'weight_decay': 0.001},
    verbose=True,
    metrics={'S_nmad': sigma_nmad, 'out_rate': out_rate}
);

Device: cuda:0
Test mode: True
EPOCH #1/100:
----------------------------------------
Trian losses: [-1.63229, -1.64166, -1.61785, -1.52805, -1.62678, -1.64253, -1.24648, -1.42389, -1.2921, -1.5399]
AVG train loss: -1.51915
Train S_nmad: 0.0358	Train out_rate: 0.07205	
----------------------------------------
Test losses: [-1.47205, -1.48193, -1.4877, -1.44688, -1.47872, -1.48302, -1.21171, -1.36367, -1.24616, -1.44012]
AVG test loss: -1.4112
Test S_nmad: 0.036	Test out_rate: 0.07303	
----------------------------------------
Time: 61.75
EPOCH #2/100:
----------------------------------------
Trian losses: [-1.64893, -1.63088, -1.60838, -1.53203, -1.62755, -1.66089, -1.267, -1.43825, -1.3027, -1.55625]
AVG train loss: -1.52729
Train S_nmad: 0.03602	Train out_rate: 0.07184	
----------------------------------------
Test losses: [-1.49088, -1.48004, -1.46794, -1.44523, -1.49296, -1.48497, -1.2425, -1.36625, -1.26119, -1.43581]
AVG test loss: -1.41678
Test S_nmad: 0.03621	Test out_rate: 0.07

In [ ]:
print(f'Benchmark S_nmad:\t{round(sigma_nmad(train_20_35_02["Z"], train_20_35_02["max35_z"]), 5)}')
print(f'Benchmark out_rate:\t{round(out_rate(train_20_35_02["Z"], train_20_35_02["max35_z"]), 5)}')

In [ ]:
# контролируемая рандомность
# нормализация, дроп ауты
# наглядные метрики

In [ ]:
# dfs = []

# feas = pd.read_pickle(
#     f'{data_dir}/20_2-fold-cv/cv2_0/part-00000.features.gz_pkl',
#     compression='gzip'
# )[['ra', 'dec', 'zspec'] + features_list]
# preds = pd.read_pickle(
#     f'{data_dir}/20_2-fold-cv/cv2_0/part-00000.predictions.x1cv2_0.gz_pkl',
#     compression='gzip'
# )[[
#     'zoo_x1cv2_019_z_max', 'zoo_x1cv2_019_z_maxConf',
#     'zoo_x1cv2_021_z_max', 'zoo_x1cv2_021_z_maxConf',
#     'zoo_x1cv2_022_z_max', 'zoo_x1cv2_022_z_maxConf',
#     'zoo_x1cv2_035_z_max', 'zoo_x1cv2_035_z_maxConf'
# ]]
# df = feas.merge(preds, how='right', left_index=True, right_index=True)
# df.columns = ['RA', 'DEC', 'Z'] + features_list + models_cols
# df['fold'] = [0] * df.shape[0]
# dfs.append(df)

# feas = pd.read_pickle(
#     f'{data_dir}/20_2-fold-cv/cv2_1/part-00000.features.gz_pkl',
#     compression='gzip'
# )[['ra', 'dec', 'zspec'] + features_list]
# preds = pd.read_pickle(
#     f'{data_dir}/20_2-fold-cv/cv2_1/part-00000.predictions.x1cv2_1.gz_pkl',
#     compression='gzip'
# )[[
#     'zoo_x1cv2_119_z_max', 'zoo_x1cv2_119_z_maxConf',
#     'zoo_x1cv2_121_z_max', 'zoo_x1cv2_121_z_maxConf',
#     'zoo_x1cv2_122_z_max', 'zoo_x1cv2_122_z_maxConf',
#     'zoo_x1cv2_135_z_max', 'zoo_x1cv2_135_z_maxConf'
# ]]
# df = feas.merge(preds, how='right', left_index=True, right_index=True)
# df.columns = ['RA', 'DEC', 'Z'] + features_list + models_cols
# df['fold'] = [1] * df.shape[0]
# dfs.append(df)

# train_20 = pd.concat(dfs, axis=0)
# train_20.drop_duplicates(subset=['RA', 'DEC'], inplace=True, keep='last')
# train_20.sort_values(by=['RA'], inplace=True)
# train_20.reset_index(drop=True, inplace=True)
# train_20.replace(np.nan, None, inplace=True)
# train_20

In [ ]:
# train_20.to_csv(f'{data_dir}/q_photo_z_train_20.csv', index=False)